In [3]:
%pylab inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle
import re
import os
from math import ceil
import random
import itertools
from scipy.stats import pearsonr, boxcox
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.tsa.stattools
import warnings

from sklearn.metrics import silhouette_samples, silhouette_score
import matplotlib.cm as cm
from sklearn.metrics import r2_score

from scipy import spatial
from sklearn import preprocessing

''' DBSCAN, AgglomerativeClustering -probably the best results '''
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.cluster import KMeans, MiniBatchKMeans, DBSCAN, AgglomerativeClustering 

warnings.filterwarnings('ignore')

In [4]:
def invboxcox(y,lmbda):
    if lmbda == 0:
        return(np.exp(y))
    else:
        return(np.exp(np.log(lmbda*y+1)/lmbda))

In [5]:
myfile = './data/reg_bin_stat_2years.pkl'
verifyfile = "./data/verif_bin_stat_5_6_2016.pkl"

In [6]:
with open(myfile, "rb") as f:
    data_2year = pickle.load(f)
    
with open(verifyfile, "rb") as f:
    data_2month = pickle.load(f)    
    
#first row of transposed matrix contains region labels
print(data_2year.T.shape)
print(data_2month.T.shape)

##### create two dataframes - taxi and verify_taxi for train and test. columns - regions, rows - time

In [7]:
verify_time = pd.date_range('2016 May 1 00:00:00', periods = data_2month.shape[1] - 1, freq = 'h')
verify_taxi = pd.DataFrame(data_2month.T[1:, :], index = [verify_time], columns=[data_2month.T[0, :].astype(int)])

time = pd.date_range('2014 May 1 00:00:00', periods = data_2year.shape[1] - 1, freq = 'h')
taxi = pd.DataFrame(data_2year.T[1:, :], index = [time], columns=[data_2year.T[0, :].astype(int)])

##### train data: [1 may 2014, 30 april 2016]   

In [26]:
taxi.head()

In [27]:
taxi.tail()

##### Not to select appropriate model for each of 102 regions, task for this week suggest to cluster data. We should choose correct number of clusters. Before clusterization, apply standardization procedure (MinMaxScaler() and StandardScaler() gives here the same result) to train dataframe - subtract mean value and divide by dispersion - USE THIS STANDARTIZED DATA ONLY FOR CLUSTERIZATION

In [13]:
taxi_data = pd.DataFrame(MinMaxScaler().fit_transform(taxi), index = taxi.index, columns=taxi.columns)
taxi_data.loc[:, taxi_data.dtypes == np.float64] = taxi_data.loc[:, taxi_data.dtypes == np.float64].apply(pd.to_numeric,downcast='float')
taxi_data= taxi_data.round(5)

In [10]:
taxi_data.head()

##### Try to choose correct number of clustersusing silhouette coefficient. Compare results for three different clusterization methods - MiniBatchKMeans - method that is appropriate for large datasets and DBSCAN, AgglomerativeClustering - two methods that usually performe well.
##### take three month serie for clusterization

The Silhouette Coefficient is calculated using the mean intra-cluster distance (``a``) and the mean nearest-cluster distance (``b``) for each sample.  The Silhouette Coefficient for a sample is ``(b - a) / max(a, b)``.  To clarify, ``b`` is the distance between a sample and the nearest cluster that the sample is not a part of.
Note that Silhouette Coefficient is only defined if number of labels is 2 <= n_labels <= n_samples - 1.
This function returns the mean Silhouette Coefficient over all samples. To obtain the values for each sample, use :func: `silhouette_samples`. The best value is 1 and the worst value is -1. Values near 0 indicate overlapping clusters. Negative values generally indicate that a sample has been assigned to the wrong cluster, as a different cluster is more similar.

In [13]:
#from sklearn.datasets import make_blobs
max_cluster = 10

for cluster_method in (MiniBatchKMeans, AgglomerativeClustering):
    for num_clusters in range(2, max_cluster+1):

        fig, ax1 = plt.subplots(1, 1)
        fig.set_size_inches(18, 7)

        ax1.set_xlim([-0.2, 1])
        # The (n_clusters+1)*10 is for inserting blank space between silhouette
        # plots of individual clusters, to demarcate them clearly.
        ax1.set_ylim([0, taxi_data.iloc[:, :].T.shape[0] + (num_clusters + 1) * 10])

        # Initialize the clusterer with n_clusters value and a random generator
        # seed of 10 for reproducibility.
        if cluster_method == AgglomerativeClustering:
            clusterer = cluster_method(n_clusters=num_clusters)
        elif  cluster_method == DBSCAN:  
            clusterer = cluster_method(eps=5000, algorithm='auto')
        else:
            clusterer = cluster_method(n_clusters=num_clusters, random_state=10)
        
        cluster_labels = clusterer.fit_predict(taxi_data.iloc[:, :].T)
        #print(cluster_labels)

        # The silhouette_score gives the average value for all the samples. This gives a perspective into the density
        # and separation of the formed clusters
        silhouette_avg = silhouette_score(taxi_data.iloc[:, :].T, cluster_labels)
        print("For n_clusters =", num_clusters, "The average silhouette_score is :", silhouette_avg)

        # Compute the silhouette scores for each sample
        sample_silhouette_values = silhouette_samples(taxi_data.iloc[:, :].T, cluster_labels)

        y_lower = 10
        for i in range(num_clusters):
            # Aggregate the silhouette scores for samples belonging to
            # cluster i, and sort them
            ith_cluster_silhouette_values = sample_silhouette_values[cluster_labels == i]

            ith_cluster_silhouette_values.sort()

            size_cluster_i = ith_cluster_silhouette_values.shape[0]
            y_upper = y_lower + size_cluster_i

            color = cm.spectral(float(i) / num_clusters)
            ax1.fill_betweenx(np.arange(y_lower, y_upper),
                              0, ith_cluster_silhouette_values,
                              facecolor=color, edgecolor=color, alpha=0.7)

            # Label the silhouette plots with their cluster numbers at the middle
            ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

            # Compute the new y_lower for next plot
            y_lower = y_upper + 10  # 10 for the 0 samples

        ax1.set_title("The silhouette plot for the various clusters.")
        ax1.set_xlabel("The silhouette coefficient values")
        ax1.set_ylabel("Cluster label")

        # The vertical line for average silhouette score of all the values
        ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

        ax1.set_yticks([])  # Clear the yaxis labels / ticks
        ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

        d = {MiniBatchKMeans:"MiniBatchKMeans", AgglomerativeClustering:"AgglomerativeClustering"}
        plt.suptitle(("Silhouette analysis for " + d[cluster_method] + " clustering on sample data "
                      "with n_clusters = %d" % num_clusters),
                     fontsize=14, fontweight='bold')

        plt.show()

##### the result look pure for all three methods. 

##### it may be a right choise to use special methods developed for clustering time series f.e. -  k-Shape method  - a new fast and accurate unsupervised time-series cluster algorithm
#### https://github.com/Mic92/kshape or https://github.com/alexminnaar/time-series-classification-and-clustering
##### it was hard to choose correct number of clusters using Silhouette Coefficient from graphs and calculation above, so 6 clusters were choosen. (why so - look at code in supplementary materials at the end of this notebook)
##### for futher calculation I take method describe here -  https://github.com/Mic92/kshape
##### clusterization process takes a lot of time for long time series so it was decided to take one month long time serie - may 2014

In [6]:
from kshape.core import kshape, zscore

In [31]:
''' time_series = [[1,2,3,4], [0,1,2,3], [0,1,2,3], [1,2,2,3]] '''
cluster_num = 6
clusters = kshape(zscore(list(taxi_data.T.values[:, :744*3]), axis=1), cluster_num)

In [ ]:
for i in range(len(clusters)):
    print(len(clusters[i][1]))

In [234]:
for i in clusters:
    print(i[1])

##### [9, 37, 48, 59, 60, 68, 70, 86] <br> [7, 11, 12, 13, 14, 15, 21, 27, 28, 31, 41, 61, 71, 72, 73, 84] <br> [74, 76, 77, 79, 80, 81, 82, 83, 85, 87, 88, 91] <br> [0, 3, 4, 5, 6, 17, 18, 19, 32, 33, 34, 35, 36, 44, 45, 95, 96, 97, 98, 99, 100, 101] <br> [8, 16, 24, 25, 26, 29, 30, 38, 39, 40, 51, 52, 62, 63, 64, 65, 75, 89, 90, 92] <br> [1, 2, 10, 20, 22, 23, 42, 43, 46, 47, 49, 50, 53, 54, 55, 56, 57, 58, 66, 67, 69, 78, 93, 94]

##### clusters object: [(np.array(centroid_coords), [..., idx, ...]),  (...),  (...)] - list of tuples, each tuple represent one cluter - numpy array of centroid coordinates followed by list of time serie indicies which belong to this cluster
##### plot clusters centroids

In [235]:
plt_num = 6
f, axarr = plt.subplots(int(np.ceil(plt_num / 3.0)), 3)
f.set_size_inches(15, 10)
for i in range(6):
    #print(i%4, i//4)
    axarr[i//3, i%3].plot(clusters[i][0])
    axarr[i//3, i%3].set_title('cluster: ' + str(i))
f.subplots_adjust(hspace=0.3)
plt.grid()
plt.show()

##### plot time serie each of 102 region to РІetermine whether they lookС‹ similar 

In [237]:
for clst_num, cluster in enumerate(clusters):
    f, axarr = plt.subplots(int(np.ceil(len(cluster[1]) / 3.0)), 3)
    f.set_size_inches(15, 5*int(np.ceil(len(cluster[1]) / 3.0)))
    for idx, graph in enumerate(cluster[1]):
        #if list(taxi_data.T.values[cluster[i], :10]) in [i[:10] for i in clust_centroids]:
        #    axarr[i//3, i%3].plot(taxi_data.T.values[cluster[i], :744], color="red") 
        axarr[idx//3, idx%3].plot(taxi_data.T.values[graph, :744])
        axarr[idx//3, idx%3].set_title('cluster: ' + str(clst_num) + " region index: " + str(graph))
    f.subplots_adjust(hspace=0.3)
    plt.show()

In [9]:
def plot_silhouette_coeff(num_clusters, cluster_labels):
    fig, ax1 = plt.subplots(1, 1)
    fig.set_size_inches(18, 7)

    ax1.set_xlim([-0.3, 1])
    # The (n_clusters+1)*10 is for inserting blank space between silhouette plots of individual clusters, to demarcate them clearly.
    ax1.set_ylim([0, taxi_data.iloc[:, :744].T.shape[0] + (num_clusters + 1) * 10])

    ''' cluster_labels already exists - see  '''
    ''' The silhouette_score gives the average value for all the samples. This gives a perspective into the density
        and separation of the formed clusters '''
    silhouette_avg = silhouette_score(taxi_data.iloc[:, :744].T, cluster_labels)
    print("For n_clusters =", num_clusters, "The average silhouette_score is :", silhouette_avg)

    '''' Compute the silhouette scores for each sample '''
    sample_silhouette_values = silhouette_samples(taxi_data.iloc[:, :744].T, cluster_labels)

    y_lower = 10
    cluster_bad_elements = {}
    for i in range(num_clusters):
        ''' Aggregate the silhouette scores for samples belonging to cluster i, and sort them '''
        ith_cluster_silhouette_values = sample_silhouette_values[cluster_labels == i]
        
        ''' select negative silhouette values for each cluster'''
        pos = [True if ((num < 0) and (cluster_labels[idx] == i)) else False for idx, num in enumerate(sample_silhouette_values)]
        cluster_bad_elements[i] = np.where(np.array(pos) == True)[0]
        
        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.spectral(float(i) / num_clusters)
        ax1.fill_betweenx(np.arange(y_lower, y_upper),
                          0, ith_cluster_silhouette_values,
                          facecolor=color, edgecolor=color, alpha=0.7)

        ''' Label the silhouette plots with their cluster numbers at the middle '''
        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        ''' Compute the new y_lower for next plot '''
        y_lower = y_upper + 10  # 10 for the 0 samples

    ax1.set_title("The silhouette plot for the various clusters.")
    ax1.set_xlabel("The silhouette coefficient values")
    ax1.set_ylabel("Cluster label")

    ''' The vertical line for average silhouette score of all the values '''
    ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

    ax1.set_yticks([])  # Clear the yaxis labels / ticks
    ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

    plt.show()
    return cluster_bad_elements

##### construct labels as array with 102 elements, each represents label of a class - e.g. [0, 0, 0, 1, 1, 2, 3, ...] 

In [10]:
''' clust_lst = [(np.array(centroid_coords), [...indexes, ...]), (...), (...)] '''
def get_centr_label(clust_lst):
    cluster_labels = np.array([0]*102)
    for counter, cluster_ids in enumerate(clust_lst):
        cluster_labels[cluster_ids[1]] = counter
    return cluster_labels    

In [19]:
get_centr_label(clusters)

In [30]:
plot_silhouette_coeff(num_clusters=10, cluster_labels=get_centr_label(clusters))

##### visual result looks much better that plot of silhouette coefficient. Maybe reason for this is that euclidean metrics was used for silhouette coefficient but k-shape method use other metrics 

##### save centroids and region indexes of each cluster since the result of clustering may vary after restart (the result is not 100% reproducible)  

In [15]:
with open('./data/yellow_cab_4_week_kshape_clust.pkl', 'wb') as f:
    pickle.dump(clusters, f)

In [16]:
with open('./data/yellow_cab_4_week_kshape_clust.pkl', "rb") as f:
    kshape_cluster = pickle.load(f)

In [42]:
print([clust for centroid, clust in kshape_cluster])

In [ ]:
cluster = [[9, 37, 48, 59, 60, 68, 70, 86], 
           [7, 11, 12, 13, 14, 15, 21, 27, 28, 31, 41, 61, 71, 72, 73, 84], 
           [74, 76, 77, 79, 80, 81, 82, 83, 85, 87, 88, 91], 
           [0, 3, 4, 5, 6, 17, 18, 19, 32, 33, 34, 35, 36, 44, 45, 95, 96, 97, 98, 99, 100, 101], 
           [8, 16, 24, 25, 26, 29, 30, 38, 39, 40, 51, 52, 62, 63, 64, 65, 75, 89, 90, 92], 
           [1, 2, 10, 20, 22, 23, 42, 43, 46, 47, 49, 50, 53, 54, 55, 56, 57, 58, 66, 67, 69, 78, 93, 94]]

In [252]:
for i in kshape_cluster[0][1]:
    if (kshape_cluster[0][0] == taxi_data.T.values[i, :744]).all():
        print(i)

##### since centriod is not one of 102 regions time serie, for each of six clusters we will find the time serie that is most correlated with the centroid and build model for this data. Six clusters - six centroids - six real time series that have the biggest correlation with centroid - six separate prediction models. We still work with first month - may 2014 - use of a two-year time sequence С€С‹ very resource demanding. 

In [5]:
def find_corr_region(kshape_cluster):
    best_corr_region = []
    all_cor = []
    for centroid, clst_idx in kshape_cluster:
        max_corr, best_region = 0, None
        corr_d = {}
        for idx in clst_idx:
            corr = pearsonr(centroid, taxi_data.T.values[idx, :744]) 
            if corr[0] > max_corr:
                max_corr = corr[0]
                best_region = idx
            corr_d[idx] = corr[0]
        all_cor.append(corr_d)    
        print("correlation with centroid: ", max_corr, " best region: ", best_region)        
        best_corr_region.append(best_region) 
    return best_corr_region, all_cor    

In [40]:
best_regions, _ = find_corr_region(kshape_cluster=kshape_cluster)

##### correlation with centroid:  0.9690473254434602  best region:  48 <br> correlation with centroid:  0.8945134166302662  best region:  21 <br> correlation with centroid:  0.7650025568694776  best region:  74 <br> correlation with centroid:  0.9393446220897997  best region:  36 <br> correlation with centroid:  0.9249631166701594  best region:  30 <br> correlation with centroid:  0.916571900401084  best region:  1

##### for each of this regions will be built and adjusted its own model and than this model will be applied to all regions that belong to the cluster 

##### some usefull functions

In [8]:
''' y = W.dot(X.T) '''
def prediction(model, components, feature_num, dataframe):
    coefs = np.array([model.params[coef] for coef in model.params.index])
    fourier_components = np.hstack((np.array([1]*dataframe.shape[0]).reshape(dataframe.shape[0],1), 
                                    dataframe.iloc[:, 1:feature_num*components+1].values))
    return coefs.dot(fourier_components.T)

In [9]:
def plot_autocorr(data, lag):
    plt.figure(figsize(15,10))
    ax = plt.subplot(211)
    sm.graphics.tsa.plot_acf(data.squeeze(), lags=lag, ax=ax)
    pylab.show()
    ax = plt.subplot(212)
    sm.graphics.tsa.plot_pacf(data.squeeze(), lags=lag, ax=ax)
    pylab.show()

In [10]:
def find_best_arima(resids, simple_dif, seson_diff, parameters_list):
    %%time
    residuals_results = []
    residuals_best_aic = float("inf")
    warnings.filterwarnings('ignore')
    resid_wrong_param = 0
    models = {}

    for counter, param in enumerate(parameters_list):
        if counter % 10 == 0:
            print(counter, end=' ')
        #some of the parameters set are incompatible
        try:
            model=sm.tsa.statespace.SARIMAX(resids, order=(param[0], simple_dif, param[1]), 
                                            seasonal_order=(param[2], seson_diff, param[3], 24)).fit(disp=-1)
            models[param] = model
        #print failure parameters and continue
        except np.linalg.LinAlgError:
            print("singular matrix: ", param)
            continue
        except ValueError:
            resid_wrong_param += 1
            continue
        aic = model.aic
        #save best model, aic, parameters
        if aic < residuals_best_aic:
            residuals_best_model = model
            residuals_best_aic = aic
            residuals_best_param = param
        residuals_results.append([param, model.aic])

    warnings.filterwarnings('default')
    return residuals_results, models

In [11]:
def plot_result(regres_res, arima_res, true_val, start_h, end_h, label):
    plt.figure(figsize=(15,5))
    plt.plot(time[start_h:end_h], regres_res[start_h:end_h] + arima_res[start_h:end_h], alpha = 0.7, color="blue", label='predicted')
    plt.plot(time[start_h:end_h], true_val[start_h:end_h].values, alpha=0.5, label = 'real value', color="red")
    plt.title(label)
    plt.xlabel('time (hour)')
    plt.ylabel("taxi calls")
    plt.grid()
    plt.legend() 
    plt.show()

In [12]:
''' train model and plot results to compare with original time serie '''
def all_cluster_model(regres, arima, regions, save_path=None):
    ''' regress : ((period, component_num), ...) '''
    ''' arima: (p,q,P,Q,d,D) '''
    ''' regions: list of column numbers in taxi frame, coresponds to cluster '''
    ''' if save_path - pickle file '''
    result = []
    for region in regions:
        print("taxi column: ", region, " region: ", taxi.columns[region], end=" ")
        taxi_reg = pd.DataFrame(taxi.iloc[:, region].values, index = taxi.index, columns=["taxi_call_num"])

        time = np.arange(0, taxi.shape[0])
        
        for regress_components in regres:
            for w in range(1, regress_components[1]+1):
                taxi_reg["sin_w_%d_%d" % (regress_components[0], w)] = np.sin(2*np.pi*w*time/regress_components[0]) 
                taxi_reg["cos_w_%d_%d" % (regress_components[0], w)] = np.cos(2*np.pi*w*time/regress_components[0])
        
        taxi_reg = taxi_reg.round(3)
        
        f_com = list(map(int, [0] + list(list(zip(*regres))[1])))
        s = 'taxi_call_num ~ ' + ' + '.join([' + '.join(list(taxi_reg.columns)[1+2*sum(f_com[:i+1]): 1+2*sum(f_com[:i+2])]) 
                                                                               for i in range(len(regres))])
        try:
            m = smf.ols(s, data=taxi_reg)
            model = m.fit(cov_type='HC1')
            pred = prediction(model, components=sum(f_com), feature_num=2, dataframe=taxi_reg)
            #residual_val = model.resid.values.astype(float32).round(3)
            try:
                resid_model=sm.tsa.statespace.SARIMAX(model.resid.values, 
                                                      order=(arima[0], arima[4], arima[1]), 
                                                 seasonal_order=(arima[2], arima[5], arima[3], 24)).fit(disp=-1)
                if save_path == None:
                    result.append(pred + resid_model.fittedvalues)
                    #plot_result(pred, resid_model.fittedvalues, taxi_reg.taxi_call_num, 0, 745, "may_2014")   
                    #plot_result(pred, resid_model.fittedvalues, taxi_reg.taxi_call_num, 3672, 4416, "october_2014")
                    #plot_result(pred, resid_model.fittedvalues, taxi_reg.taxi_call_num, 13920, 14664, "december_2015")
                else:
                    #path:'H:/Yandex machine learning/finall course coursera/
                    resid_model.save(path + "region_%d"%(taxi_data.columns[region]) + "_sarimax.pkl") 
                    with open(path + "region_%d"%(taxi_data.columns[region]) + "_regression.pkl", 'wb') as f:
                        pickle.dump(model, f, protocol=pickle.HIGHEST_PROTOCOL)
            
            except ValueError as e:
                print(region, e)
                try:
                    log_resid, lmbda = boxcox(abs(min(model.resid.values)) + 0.1 + model.resid.values)
                    resid_model=sm.tsa.statespace.SARIMAX(log_resid, 
                                                          order=(arima[0], arima[4], arima[1]), 
                                                     seasonal_order=(arima[2], arima[5], arima[3], 24)).fit(disp=-1)

                    if save_path == None:
                        resid_prediction = resid_model.fittedvalues
                        resid_prediction[resid_prediction == 0] = 0.01
                        resid_prediction = invboxcox(resid_prediction, lmbda)
                        resid_prediction = resid_prediction - abs(min(model.resid.values)) - 0.1

                        result.append(pred + resid_model.fittedvalues)
                        #plot_result(pred, resid_prediction, taxi_reg.taxi_call_num, 0, 745, "may_2014")   
                        #plot_result(pred, resid_prediction, taxi_reg.taxi_call_num, 3672, 4416, "october_2014")
                        #plot_result(pred, resid_prediction, taxi_reg.taxi_call_num, 13920, 14664, "december_2015")
                    else:
                        #path:'H:/Yandex machine learning/finall course coursera/
                        resid_model.save(path + "region_%d"%(taxi_data.columns[region]) + "log_sarimax.pkl") 
                        with open(path + "region_%d"%(taxi_data.columns[region]) + "_regression.pkl", 'wb') as f:
                            pickle.dump(model, f, protocol=pickle.HIGHEST_PROTOCOL)
                   
                except ValueError as e:
                    print(region, e)
                    print("unfortunally boxcox does not help")
                        
        #except (ValueError, LinAlgError) as e:
        except LinAlgError as e:
            print(region, e)
            continue  
    return result        

In [13]:
def make_prediction(regres, arima, region, prediction_period=('2016-05-01 00:00:00', '2016-05-31 23:00:00'), draw=True):
    ''' regress : ((period, component_num), ...) '''
    ''' arima: (p,q,P,Q,d,D) '''
    ''' regions: list of column numbers in taxi frame, coresponds to cluster '''
    ''' prediction_period: '''
    
    ''' regres=((168, 80), (8760, 15), (2, 23)), arima=(2, 1, 1, 1, 1, 0) '''
    
    print("taxi column: ", region, " region: ", taxi.columns[region], end=" ")
    taxi_reg = pd.DataFrame(taxi.iloc[:, region].values, index = taxi.index, columns=["taxi_call_num"])
    time = np.arange(0, taxi.shape[0])
    
    with open("H:/Yandex machine learning/finall course coursera/verif_bin_stat_5_6_2016.pkl", "rb") as f:
            data_2month_verif = pickle.load(f)

    index = pd.DatetimeIndex(start=prediction_period[0],end=prediction_period[1],freq='1h')
    future_prediction = pd.DataFrame(data_2month_verif[region, 1:index.shape[0]+1], index=index, columns=["taxi_call_num"])
    future_time = np.arange(0, future_prediction.shape[0])
    
    
    taxi_reg = pd.concat([taxi_reg, future_prediction], axis=0)
      
    for regress_components in regres:
        for w in range(1, regress_components[1]+1):
            taxi_reg["sin_w_%d_%d" % (regress_components[0], w)] = np.sin(2*np.pi*w*np.concatenate((time, future_time))/regress_components[0]) 
            taxi_reg["cos_w_%d_%d" % (regress_components[0], w)] = np.cos(2*np.pi*w*np.concatenate((time, future_time))/regress_components[0])
    taxi_reg = taxi_reg.round(3)

    f_com = list(map(int, [0] + list(list(zip(*regres))[1])))
    s = 'taxi_call_num ~ ' + ' + '.join([' + '.join(list(taxi_reg.columns)[1+2*sum(f_com[:i+1]): 1+2*sum(f_com[:i+2])]) for i in range(len(regres))])
    
    
    def draw_result():
        plt.figure(figsize(15,7))
        plt.plot(taxi_reg.index.values[-index.shape[0]-24*2:], taxi_reg.taxi_call_num[-index.shape[0]-24*2:], alpha = 0.5, color="red", label='real value')
        plt.plot(taxi_reg.index.values[-index.shape[0]-24*2:], (taxi_reg["regression_prediction"] + taxi_reg["average_resid_pred"])[-index.shape[0]-24*2:], alpha = 0.7, color="blue", label='predicted')
        plt.title("using past data")
        plt.show()  
        plt.figure(figsize(15,7))
        plt.plot(taxi_reg.index.values[-index.shape[0]-24*2:], taxi_reg.taxi_call_num[-index.shape[0]-24*2:], alpha = 0.5, color="red", label='real value')
        plt.plot(taxi_reg.index.values[-index.shape[0]-24*2:], (taxi_reg["regression_prediction"] + taxi_reg["resid_prediction"])[-index.shape[0]-24*2:], alpha = 0.7, color="blue", label='predicted')
        plt.title("using arima prediction")
        plt.show() 
    
    
    try:
        m = smf.ols(s, data=taxi_reg.iloc[:time.shape[0], :])
        model = m.fit(cov_type='HC1')
        pred = prediction(model, components=sum(f_com), feature_num=2, dataframe=taxi_reg)
        
        taxi_reg["regression_prediction"] = pred
               
        ''' first variant '''
        ''' ASSUME THAT THE FUTURE RESULT IN UNKNOWN, SO WE USE RESIDUALS FOR THE SAME PERIOD OF THE PAST YEAR 
            FOR ARIMA RESIDUAL PREDICTION  '''
                
        start_2015 = np.where(taxi_reg.index==(index[0] - pd.DateOffset(years=1)))[0][0]  
        end_2015 = np.where(taxi_reg.index==(index[-1] - pd.DateOffset(years=1)))[0][0]
        start_2014 = np.where(taxi_reg.index==(index[0] - pd.DateOffset(years=2)))[0][0]  
        end_2014 = np.where(taxi_reg.index==(index[-1] - pd.DateOffset(years=2)))[0][0]
                
        try:
            resid_model=sm.tsa.statespace.SARIMAX(model.resid.values, 
                                                  order=(arima[0], arima[4], arima[1]), 
                                             seasonal_order=(arima[2], arima[5], arima[3], 24)).fit(disp=-1)
            taxi_reg["average_resid_pred"] = np.concatenate((resid_model.fittedvalues, 
                                                            (resid_model.fittedvalues[start_2014:end_2014 + 1] + 
                                                             resid_model.fittedvalues[start_2015:end_2015 + 1])/2))
            
            taxi_reg["resid_prediction"] = np.concatenate((resid_model.fittedvalues, 
                                                        resid_model.predict(start=time.shape[0], 
                                                                            end=time.shape[0] + future_time.shape[0]-1, 
                                                                            dynamic=True)))
            if draw:
                draw_result()
            
            return taxi_reg[["regression_prediction", "average_resid_pred", "resid_prediction"]]
            
        except ValueError as e:
            print(region, e)
            try:
                log_resid, lmbda = boxcox(abs(min(model.resid.values)) + 0.1 + model.resid.values)
                resid_model=sm.tsa.statespace.SARIMAX(log_resid, 
                                                      order=(arima[0], arima[4], arima[1]), 
                                                 seasonal_order=(arima[2], arima[5], arima[3], 24)).fit(disp=-1)

                resid_prediction = resid_model.fittedvalues
                resid_prediction[resid_prediction == 0] = 0.01
                resid_prediction = invboxcox(resid_prediction, lmbda)
                resid_prediction = (resid_prediction - abs(min(model.resid.values)) - 0.1)
                taxi_reg["average_resid_pred"] = np.concatenate((resid_prediction, 
                                                            (resid_prediction[start_2014:end_2014 + 1] + 
                                                             resid_prediction[start_2015:end_2015 + 1])/2))
                
                future_resid = invboxcox(
                         resid_model.predict(start=time.shape[0], end=time.shape[0] + future_time.shape[0]-1, dynamic=True), lmbda)                                           
                taxi_reg["resid_prediction"] = np.concatenate((resid_prediction, future_resid - abs(min(model.resid.values)) - 0.1))                                           

                if draw:
                    draw_result()
                
                return taxi_reg[["regression_prediction", "average_resid_pred", "resid_prediction"]]
            
            except ValueError as e:
                print(region, e)
                print("unfortunally boxcox does not help") 
                
        #plot_result(pred, resid_prediction, taxi_reg.taxi_call_num, 0, 745, "may_2016")
    
    except LinAlgError as e:
        print(region, e)               

In [14]:
''' find best fourier components and number of this components by R2 coeff and degree of correlation with typical serie lags '''
def find_best_fourier(region, fourier_lst, ):
    """ region - num of column """
    """ fourier_lst - ((number_of components, period), (number_of components, period), ) """
    
    clst = pd.DataFrame(taxi.iloc[:, region].values, index = taxi.index, columns=["taxi_call_num"])
    time = np.arange(0, taxi.shape[0])
    
    
    for fourier_components in fourier_lst:
            for w in range(1, fourier_components[0]+1):
                clst["sin_w_%d_%d" % (fourier_components[1], w)] = np.sin(2*np.pi*w*time/fourier_components[1]) 
                clst["cos_w_%d_%d" % (fourier_components[1], w)] = np.cos(2*np.pi*w*time/fourier_components[1])
    
    
    models = []
    f_com = list(map(int, [0] + list(list(zip(*fourier_lst))[0])))
    s = 'taxi_call_num ~ ' + ' + '.join([' + '.join(list(clst.columns)[1+2*sum(f_com[:i+1]): 1+2*sum(f_com[:i+2])]) for i in range(len(fourier_lst)-1)])
        
    for W in range(1, f_com[-1]+1):
        try:
            current_s = s + ' + ' + ' + '.join(list(clst.columns)[1+2*sum(f_com[:len(fourier_lst)]): 1+2*sum(f_com[:len(fourier_lst)]) +2*W])
            m = smf.ols(current_s, data=clst)
        except ValueError:
            continue
        
        model = m.fit(cov_type='HC1')
        models.append(model)  
            
    fig1 = figure(figsize=(15,4))
 
    ax1 = fig1.add_subplot(111)
    line1 = ax1.plot(range(1, f_com[-1] + 1), list(map(lambda x: x.rsquared, models)), color="blue", label="r-square")
    ylabel("r-square")

    ax2 = fig1.add_subplot(111, sharex=ax1, frameon=False)
    line2 = ax2.plot(range(1, f_com[-1] + 1), list(map(lambda x: x.mse_resid, models)), color="green", label="resid")
    ax2.yaxis.tick_right()
    ax2.yaxis.set_label_position("right")
    ylabel("resid")

    legend((line1, line2), ("1", "2"))
    legend()
    grid()
    show()    
    
    plt.figure(figsize=(15,4))
    plt.plot(range(1, f_com[-1] + 1), [statsmodels.tsa.stattools.acf(x=model.resid, nlags=168)[168] for model in models], color="blue", label='week')
    plt.plot(range(1, f_com[-1] + 1), [statsmodels.tsa.stattools.acf(x=model.resid, nlags=365 * 24)[365 * 24] for model in models], color="green", label='year')
    plt.plot(range(1, f_com[-1] + 1), [statsmodels.tsa.stattools.acf(x=model.resid, nlags=168)[24] for model in models], color="red", label='day')
    plt.grid()
    plt.legend()
    plt.xlabel("number of fourier components")
    plt.ylabel("autocorellation with 168, 24, 365*24 lags")
    plt.show()

In [15]:
def prediction_real(df_1, df_2, segment, title, ylabel, size, labels):
    plt.figure(figsize=size)
    plt.plot(df_1[segment[0]: segment[1]], alpha = 0.7, label = labels[0])
    plt.plot(df_2[segment[0]: segment[1]], alpha=0.5, label = labels[1], color="red")
    plt.xlabel('time (hours)')
    plt.ylabel(ylabel)
    plt.title(title)
    plt.legend()
    plt.grid()    

### Start to create models

##### 1) Cluster 0, region - column number 48 in taxi dataframe

In [13]:
zero_clst = pd.DataFrame(taxi.iloc[:, 48].values, index = taxi.index, columns=["taxi_call_num"])
zero_clst.head()

##### look at entire time serie of region

In [20]:
fig, axes = plt.subplots(figsize = (150, 15))
axes.title.set_size(40)
zero_clst.plot(color="green", title="timeserie", fontsize=25, ax = axes)
plt.show()
print(zero_clst.shape)

#double click on image makes it bigger

##### the annual seasonality is clearly visible

##### look at first three months

In [22]:
month_len = np.array([0, 744, 720, 744])

for i, c in enumerate(["May", "June", "July"]):
    zero_clst[int(sum(month_len[:i+1])) : int(sum(month_len[:i+2]))].plot(figsize=(20, 5), title=c, color="green", fontsize=16, grid=True)
    plt.show()

In [15]:
plt.figure(figsize(15,10))
sm.tsa.seasonal_decompose(zero_clst[:2208]).plot()
print("DickeyвЂ“Fuller test: p=%f" % sm.tsa.stattools.adfuller(zero_clst.taxi_call_num[:2208].values)[1])

##### weekly seasonality is clearly visible

##### ARIMA model does not work with too big periods (more than 350)- like annual seasonality in our case also it is slow for long time series and working with large periods ARIMA is requires a lot of RAM. So we will apply ARIMA to residuals of linear regression prediction model.

##### ARIMA use a lot of memory and works too long even with smallest of three periodocities (annual weekly daily). Each of this periodicities will be taken into account by linear regression model. 

##### using find_best_fourier function we may consistently choose number of each fourier components - to get best score and minimum correlation with seasonal lags

In [14]:
TT = 365 * 24
Tt = 7 * 24
tt = 2
time = np.arange(0, taxi.shape[0])
for w_week in range(1, 81):
    zero_clst["sin_week_" + str(w_week)] = np.sin(2*np.pi*w_week*time/Tt) 
    zero_clst["cos_week_" + str(w_week)] = np.cos(2*np.pi*w_week*time/Tt)
for w_year in range(1, 16):
    zero_clst["sin_year_" + str(w_year)] = np.sin(2*np.pi*w_year*time/TT) 
    zero_clst["cos_year_" + str(w_year)] = np.cos(2*np.pi*w_year*time/TT) 
for w_day in range(1, 31):
    zero_clst["sin_day_" + str(w_day)] = np.sin(2*np.pi*w_day*time/tt)
    zero_clst["cos_day_" + str(w_day)] = np.cos(2*np.pi*w_day*time/tt)  

In [20]:
zero_clst.shape

In [26]:
find_best_fourier(48, ((80, 168), (15, 8760), (31, 2), ))

In [21]:
#models = []
#for W in range(1, 31):
#    m1 = smf.ols('taxi_call_num ~ ' + ' + '.join(list(zero_clst.columns)[1: 2*80+1]) + ' + ' + 
#                 ' + '.join(list(zero_clst.columns)[2*80 + 1: 2*80 + 1 + 2*15]) + ' + ' + 
#                 ' + '.join(list(zero_clst.columns)[2*80 + 1 + 2*15: 2*80 + 1 + 2*15 + 2*W]), data=zero_clst)
#    model = m1.fit(cov_type='HC1')
#    models.append(model)
    
#models[23].summary()
# fitted.summary(); fitted.params; models[-1].rsquared; params.Intercept  

##### it will be enought to choose  80 components with week period (Tt=168), 15 year components (TT=365*24) and 23 hour prediods (tt=2). Ech component consist of 2 elements sin and cos => (80+15+23)*2

In [16]:
m1 = smf.ols('taxi_call_num ~ ' + ' + '.join(list(zero_clst.columns)[1: 2*80+1]) + ' + ' + \
                                  ' + '.join(list(zero_clst.columns)[2*80 + 1: 2*80 + 1 + 2*15]) + ' + ' + \
                                  ' + '.join(list(zero_clst.columns)[2*80 + 1 + 2*15: 2*80 + 1 + 2*15 + 2*23]), data=zero_clst)

final_model_clust_0 = m1.fit(cov_type='HC1')

In [17]:
pred = prediction(final_model_clust_0, components=118, feature_num=2, dataframe=zero_clst)

In [27]:
prediction_real(pred, zero_clst.taxi_call_num.values, (0, 745), "May 2014", 'taxi calls', (15,5), ('prediction', 'real values'))

##### residuals

In [30]:
plt.figure(figsize=(20,6))
plt.plot(final_model_clust_0.resid)
plt.title("residual plot")
plt.xlabel('time (hour)')
plt.ylabel('residuals')
plt.grid()
plt.show()

##### there is no annual, month seasonal periodicity

##### look at three monyj residuals

In [29]:
plt.figure(figsize(15,10))
sm.tsa.seasonal_decompose(final_model_clust_0.resid[:2209]).plot()
print("DickeyвЂ“Fuller test: p=%f" % sm.tsa.stattools.adfuller(final_model_clust_0.resid[:4209].values)[1])

##### build ARIMA model at residual values

In [18]:
resids = pd.DataFrame(final_model_clust_0.resid.values, index = [time], columns=["residuals"])
#resids.loc[:, resids.dtypes == np.float64] = resids.loc[:, resids.dtypes == np.float64].apply(pd.to_numeric,downcast='float')
resids= resids.round(3)

In [32]:
resids.head()

In [34]:
plot_autocorr(resids.residuals, lag=168)

##### daily seasonality remained

In [20]:
resids["diff_1"] = resids.residuals - resids.residuals.shift(1)

In [36]:
plot_autocorr(resids["diff_1"][1:].values, lag=168)

In [37]:
''' Autocorrelation - РїР°СЂР°РјРµС‚СЂС‹ q, Q '''
ps = range(0, 14)
qs = range(0, 4)
Ps = range(0, 2)
Qs = range(0, 2)

In [16]:
parameters = itertools.product(ps, qs, Ps, Qs)
parameters_list = list(parameters)
len(parameters_list)

In [20]:
type(resids.diff_24[30].values[0])

##### choose best model on first 6 month data

In [17]:
residuals_results, models = find_best_arima(resids.residuals.values[0:744*6], simple_dif=1, seson_diff=0, parameters_list=parameters_list[30:])

In [18]:
print(sorted(residuals_results, key=lambda tup: tup[1], reverse=False)[:5])

five best results

[[(13, 2, 1, 1), -62230.40738180563], [(10, 2, 1, 1), -62057.27396666515], [(6, 1, 1, 1), -61984.42501707877], [(2, 1, 1, 1), -61931.73855506932], \n[(11, 3, 1, 1), -61876.288660259976]]

##### best model (13, 2, 1, 1)

In [21]:
model_1111 = sm.tsa.statespace.SARIMAX(resids.residuals.values, order=(13, 1, 1), 
                                        seasonal_order=(2, 0, 1, 24)).fit(disp=-1)

In [22]:
plt.figure(figsize=(15,8))
plt.subplot(211)
plt.plot(model_1111.resid[25:])#.plot()
plt.ylabel(u'Residuals')

ax = plt.subplot(212)
sm.graphics.tsa.plot_acf(model_1111.resid[25:].squeeze(), lags=168, ax=ax)
plt.show();

##### plot predicted and real residuals for may 2014

In [25]:
prediction_real(model_1111.fittedvalues, resids.residuals, (0, 744), "May 2014", 'residuals', (15,5), 
                ('predict residuals', 'real residuals'))

##### add ARIMA results to values fitted by linear regression - get final result. Look at fitted result for three months 

In [26]:
plot_result(pred, model_1111.fittedvalues, zero_clst.taxi_call_num, 0, 745, "may_2014")

In [27]:
plot_result(pred, model_1111.fittedvalues, zero_clst.taxi_call_num, 3672, 4416, "october_2014")

In [28]:
plot_result(pred, model_1111.fittedvalues, zero_clst.taxi_call_num, 13920, 14664, "december_2015")

##### For all regions from cluster 0 we've got linear regression model on 80 week (t=168), 15 annual (t=365*24), 23 hour (t=2) fourier componentsС‚ + ARIMA(2, 1, 1, 1) (or (13, 2, 1, 1)) for regression residuals. 

##### 2) Cluster 1, region - column number 21 in taxi dataframe

In [29]:
one_clst = pd.DataFrame(taxi.iloc[:, 21].values, index = taxi.index, columns=["taxi_call_num"])
one_clst.head()

##### look at entire time serie of region

In [30]:
fig, axes = plt.subplots(figsize = (150,15))
axes.title.set_size(40)
one_clst.plot(color="green", title="timeserie", fontsize=25, ax = axes)
plt.show()
print(one_clst.shape)

#double click on image makes it bigger

##### look at first three months

In [31]:
month_len = np.array([0, 744, 720, 744])

for i, c in enumerate(["May", "June", "July"]):
    one_clst[int(sum(month_len[:i+1])) : int(sum(month_len[:i+2]))].plot(figsize=(20, 5), title=c, color="green", fontsize=16, grid=True)
    plt.show()

In [13]:
plt.figure(figsize(15,10))
sm.tsa.seasonal_decompose(one_clst[:2208]).plot();

##### do all the same as in previous episode with cluster 0

In [32]:
TT = 365 * 24
Tt = 7 * 24
tt = 1
time = np.arange(0, taxi.shape[0])
for w_week in range(1, 71):
    one_clst["sin_week_" + str(w_week)] = np.sin(2*np.pi*w_week*time/Tt) 
    one_clst["cos_week_" + str(w_week)] = np.cos(2*np.pi*w_week*time/Tt)
for w_day in range(1, 13):
    one_clst["sin_day_" + str(w_day)] = np.sin(2*np.pi*w_day*time/tt)
    one_clst["cos_day_" + str(w_day)] = np.cos(2*np.pi*w_day*time/tt) 
for w_year in range(1, 16):
    one_clst["sin_year_" + str(w_year)] = np.sin(2*np.pi*w_year*time/TT) 
    one_clst["cos_year_" + str(w_year)] = np.cos(2*np.pi*w_year*time/TT)   

In [33]:
find_best_fourier(21, ((70, 168), (15, 8760), (30, 1)))
""" region - num of column """
""" fourier_lst - ((number_of components, period), (number_of components, period), ) """

##### it will be enought to choose 70 components with week period (Tt=168), 12 year components (TT=36524) and 15 hour prediods (tt=1). 

In [34]:
m2 = smf.ols('taxi_call_num ~ ' + ' + '.join(list(one_clst.columns)[1: 2*70+1]) + ' + ' + 
                                  ' + '.join(list(one_clst.columns)[2*70 + 1: 2*70 + 1 + 2*12]) 
                                + ' + ' +' + '.join(list(one_clst.columns)[2*70 + 1 + 2*12: 2*70 + 1 + 2*12 + 2*15]), data=one_clst)
model = m2.fit(cov_type='HC1')
final_model_clust_1 = model

In [35]:
plt.figure(figsize=(20,5))
plt.plot(final_model_clust_1.resid)
plt.title("residual plot")
plt.xlabel('time (hour)')
plt.ylabel('residuals')
plt.grid()
plt.show()

In [36]:
resids1 = pd.DataFrame(final_model_clust_1.resid.values, index = [time], columns=["residuals"]).round(2)
resids1.head()

In [37]:
plot_autocorr(resids1.residuals, lag=168)

In [38]:
resids1["diff_1"] = resids1.residuals - resids1.residuals.shift(1)

In [39]:
plot_autocorr(resids1["diff_1"][1:], lag=168)

In [16]:
''' Autocorrelation - РїР°СЂР°РјРµС‚СЂС‹ q, Q '''
ps = range(0, 8)
qs = range(0, 14)
Ps = range(0, 2)
Qs = range(0, 2)

In [17]:
parameters = itertools.product(ps, qs, Ps, Qs)
parameters_list = list(parameters)
len(parameters_list)

In [18]:
residuals_results, models = find_best_arima(resids1.residuals.values[0:744*6], simple_dif=1, seson_diff=0, parameters_list=parameters_list[300:])

In [19]:
print(sorted(residuals_results, key=lambda tup: tup[1], reverse=False)[:5])

best models

[[(5, 1, 1, 1), 47675.95240058894], [(6, 1, 1, 1), 47677.91827414791], [(7, 1, 1, 1), 47681.338857963274], [(6, 12, 1, 1), 47692.614959403974], [(6, 2, 1, 1), 47693.0237124409]]

##### best model (5, 1, 1, 1)

In [40]:
best_model = sm.tsa.statespace.SARIMAX(resids1.residuals.values, order=(5, 1, 1), 
                                        seasonal_order=(1, 0, 1, 24)).fit(disp=-1)

In [41]:
plt.figure(figsize=(15,8))
plt.subplot(211)
plt.plot(best_model.resid[1:])#.plot()
plt.ylabel(u'Residuals')

ax = plt.subplot(212)
sm.graphics.tsa.plot_acf(best_model.resid[1:].squeeze(), lags=168, ax=ax)
plt.show();

##### plot predicted and real residuals for may 2014

In [42]:
prediction_real(best_model.fittedvalues, resids.residuals, (0, 744), "May 2014", 'residuals', (15,5), 
                ('predict residuals', 'real residuals'))

##### add ARIMA results to values fitted by linear regression - get final result. Look at fitted result for two months

In [45]:
plot_result(prediction(final_model_clust_1, components=97, feature_num=2, dataframe=one_clst), 
            best_model.fittedvalues, one_clst.taxi_call_num, 0, 745, "may_2014")
plot_result(prediction(final_model_clust_1, components=97, feature_num=2, dataframe=one_clst), 
            best_model.fittedvalues, one_clst.taxi_call_num, 13920, 14664, "december_2015")

##### For all regions from cluster 1 we've got linear regression model on 70 week (t=168), 15 annual (t=365*24), 12 hour (t=1) fourier componentsС‚ + ARIMA(5, 1, 1, 1) for regression residuals. Use this model to fit all cluster regions

##### 3) Cluster 2, region - column number 74 in taxi dataframe

In [13]:
two_clst = pd.DataFrame(taxi.iloc[:, 74].values, index = taxi.index, columns=["taxi_call_num"])
two_clst.head()

In [14]:
fig, axes = plt.subplots(figsize = (150,15))
axes.title.set_size(40)
two_clst.plot(color="green", title="timeserie", fontsize=25, ax = axes)
plt.show()
print(two_clst.shape)

#double click on image makes it bigger

##### the annual seasonality is clearly visible

##### look at first three months

In [15]:
month_len = np.array([0, 744, 720, 744])

for i, c in enumerate(["May", "June", "July"]):
    two_clst[int(sum(month_len[:i+1])) : int(sum(month_len[:i+2]))].plot(figsize=(20, 5), title=c, color="green", fontsize=16, grid=True)
    plt.show()

In [16]:
plt.figure(figsize(15,10))
sm.tsa.seasonal_decompose(pd.Series(two_clst.taxi_call_num[:2208])).plot()
print("DickeyвЂ“Fuller test: p=%f" % sm.tsa.stattools.adfuller(two_clst.taxi_call_num[:2208].values)[1])

##### weekly seasonality is clearly visible

##### using find_best_fourier function we may consistently choose number of each fourier components - to get best score and minimum correlation with seasonal lags

In [17]:
find_best_fourier(74, ((70, 168), (20, 8760), (30, 1), ))
""" region - num of column """
""" fourier_lst - ((number_of components, period), (number_of components, period), ) """

In [18]:
TT = 365 * 24
Tt = 7 * 24
tt = 1
time = np.arange(0, taxi.shape[0])
for w_week in range(1, 71):
    two_clst["sin_week_" + str(w_week)] = np.sin(2*np.pi*w_week*time/Tt) 
    two_clst["cos_week_" + str(w_week)] = np.cos(2*np.pi*w_week*time/Tt)
for w_day in range(1, 13):
    two_clst["sin_day_" + str(w_day)] = np.sin(2*np.pi*w_day*time/tt)
    two_clst["cos_day_" + str(w_day)] = np.cos(2*np.pi*w_day*time/tt) 
for w_year in range(1, 21):
    two_clst["sin_year_" + str(w_year)] = np.sin(2*np.pi*w_year*time/TT) 
    two_clst["cos_year_" + str(w_year)] = np.cos(2*np.pi*w_year*time/TT)  

##### it will be enought to choose 70 components with week period (Tt=168), 20 year components (TT=36524) and 12 hour prediods (tt=1)

In [19]:
m3 = smf.ols('taxi_call_num ~ ' + ' + '.join(list(two_clst.columns)[1: 2*70+1]) + ' + ' + 
                                  ' + '.join(list(two_clst.columns)[2*70 + 1: 2*70 + 1 + 2*12]) 
                                + ' + ' +' + '.join(list(two_clst.columns)[2*70 + 1 + 2*12: 2*70 + 1 + 2*12 + 2*20]), data=two_clst)
final_model_clust_2 = m3.fit(cov_type='HC1')

In [20]:
plt.figure(figsize=(20,6))
plt.plot(final_model_clust_2.resid)
plt.title("residual plot")
plt.xlabel('time (hour)')
plt.ylabel('residuals')
plt.grid()
plt.show()

##### there is no annual, month seasonal periodicity

##### look at three month residuals

In [21]:
resids2 = pd.DataFrame(final_model_clust_2.resid.values, index = [time], columns=["residuals"]).round(2)
resids2.head()

In [22]:
plot_autocorr(resids2.residuals, lag=168)

In [23]:
resids2["diff_1"] = resids2.residuals - resids2.residuals.shift(1)

In [24]:
plot_autocorr(resids2["diff_1"][1:], lag=168)

In [25]:
''' Autocorrelation - РїР°СЂР°РјРµС‚СЂС‹ q, Q '''
ps = range(0, 18)
qs = range(0, 4)
Ps = range(0, 2)
Qs = range(0, 2)

In [18]:
parameters = itertools.product(ps, qs, Ps, Qs)
parameters_list = list(parameters)
len(parameters_list)

In [19]:
residuals_results, models = find_best_arima(resids2.residuals.values[0:744*6], simple_dif=1, seson_diff=0, 
                                            parameters_list=parameters_list[264:])

In [20]:
print(sorted(residuals_results, key=lambda tup: tup[1], reverse=False)[:5])

best models

[[(14, 1, 1, 1), 26649.113279118814], [(17, 1, 1, 1), 26652.25337989754], [(15, 2, 1, 1), 26655.680252936225], [(16, 1, 1, 1), 26655.999645383225], [(11, 1, 1, 1), 26656.231480555634], [(9, 1, 1, 1), 26657.434251443105], [(12, 2, 1, 1), 26658.540766088317], [(11, 3, 1, 1), 26659.657781788857], [(12, 1, 1, 1), 26659.975405869918], [(9, 2, 1, 1), 26660.11414850546]]

In [27]:
best_model = sm.tsa.statespace.SARIMAX(resids2.residuals.values, order=(14, 1, 1), 
                                        seasonal_order=(1, 0, 1, 24)).fit(disp=-1)

In [28]:
plt.figure(figsize=(15,8))
plt.subplot(211)
plt.plot(best_model.resid[1:])#.plot()
plt.ylabel(u'Residuals')

ax = plt.subplot(212)
sm.graphics.tsa.plot_acf(best_model.resid[1:].squeeze(), lags=168, ax=ax)
plt.show();

##### plot predicted and real residuals for may 2014

In [32]:
prediction_real(best_model.fittedvalues, resids2.residuals, (0, 744), "May 2014", 'residuals', (15,5), 
                ('predict residuals', 'real residuals'))

In [33]:
plot_result(prediction(final_model_clust_2, components=102, feature_num=2, dataframe=two_clst), 
            best_model.fittedvalues, two_clst.taxi_call_num, 0, 745, "may_2014")
plot_result(prediction(final_model_clust_2, components=102, feature_num=2, dataframe=two_clst), 
            best_model.fittedvalues, two_clst.taxi_call_num, 13920, 14664, "december_2015")

##### For all regions from cluster 2 we've got linear regression model on 70 week (t=168), 20 annual (t=365*24), 12 hour (t=1) fourier componentsС‚ + ARIMA(14, 1, 1, 1) for regression residuals. Use this model to fit all cluster regions

##### 4) Cluster 3, region - column number 36 in taxi dataframe 

In [34]:
three_clst = pd.DataFrame(taxi.iloc[:, 36].values, index = taxi.index, columns=["taxi_call_num"])
three_clst.head()

##### look at entire time serie of region

In [35]:
fig, axes = plt.subplots(figsize = (150,15))
axes.title.set_size(40)
three_clst.plot(color="green", title="timeserie", fontsize=25, ax = axes)
plt.show()
print(three_clst.shape)

#double click on image makes it bigger

##### the annual seasonality is clearly visible

##### look at first three months

In [36]:
month_len = np.array([0, 744, 720, 744])

for i, c in enumerate(["May", "June", "July"]):
    three_clst[int(sum(month_len[:i+1])) : int(sum(month_len[:i+2]))].plot(figsize=(20, 5), title=c, color="green", fontsize=16, grid=True)
    plt.show()

In [37]:
plt.figure(figsize(15,10))
sm.tsa.seasonal_decompose(three_clst.iloc[2208*0:2208*1, 0]).plot()
print("DickeyвЂ“Fuller test: p=%f" % sm.tsa.stattools.adfuller(three_clst.taxi_call_num[2208*0:2208*1].values)[1])

In [38]:
TT = 365 * 24
Tt = 7 * 24
tt = 11
time = np.arange(0, taxi.shape[0])
for w_week in range(1, 61):
    three_clst["sin_week_" + str(w_week)] = np.sin(2*np.pi*w_week*time/Tt) 
    three_clst["cos_week_" + str(w_week)] = np.cos(2*np.pi*w_week*time/Tt)
for w_year in range(1, 16):
    three_clst["sin_year_" + str(w_year)] = np.sin(2*np.pi*w_year*time/TT) 
    three_clst["cos_year_" + str(w_year)] = np.cos(2*np.pi*w_year*time/TT)
for w_day in range(1, 13):
    three_clst["sin_day_" + str(w_day)] = np.sin(2*np.pi*w_day*time/tt)
    three_clst["cos_day_" + str(w_day)] = np.cos(2*np.pi*w_day*time/tt) 

In [39]:
three_clst.shape

##### using find_best_fourier function we may consistently choose number of each fourier components - to get best score and minimum correlation with seasonal lags

In [41]:
models = []
for W in range(1, 13):
    m4 = smf.ols('taxi_call_num ~ ' + ' + '.join(list(three_clst.columns)[1: 2*60+1]) + ' + ' + 
                 ' + '.join(list(three_clst.columns)[2*60 + 1: 2*60 + 1 + 2*15]) + 
                 ' + ' + ' + '.join(list(three_clst.columns)[2*60 + 1 + 2*15: 2*60 + 1 + 2*15 + 2*W]), data=three_clst)
    model = m4.fit(cov_type='HC1')
    models.append(model)

In [42]:
find_best_fourier(36, ((60, 168), (15, 8760), (30, 11), ))

##### it will be enought to choose 60 components with week period (Tt=168), 15 year components (TT=365*24) and 12 hour prediods (tt=11).

In [43]:
m4 = smf.ols('taxi_call_num ~ ' + ' + '.join(list(three_clst.columns)[1: 2*60+1]) + ' + ' + 
                                  ' + '.join(list(three_clst.columns)[2*60 + 1: 2*60 + 1 + 2*15]) 
                                + ' + ' +' + '.join(list(three_clst.columns)[2*60 + 1 + 2*15: 2*60 + 1 + 2*15 + 2*12]), data=three_clst)
model = m4.fit(cov_type='HC1')
final_model_clust_3 = model

##### residuals

In [45]:
plt.figure(figsize=(20,5))
plt.plot(final_model_clust_3.resid)
plt.title("residual plot")
plt.xlabel('time (hour)')
plt.ylabel('residuals')
plt.grid()
plt.show()

In [46]:
resids3 = pd.DataFrame(final_model_clust_3.resid.values, index = [time], columns=["residuals"]).round(2)
resids3.head()

In [47]:
plot_autocorr(resids3.residuals, lag=168)

In [48]:
resids3["diff_1"] = resids3.residuals - resids3.residuals.shift(1)

In [49]:
plot_autocorr(resids3["diff_1"][1:], lag=168)

In [50]:
''' Autocorrelation - РїР°СЂР°РјРµС‚СЂС‹ q, Q '''
ps = range(0, 5)
qs = range(0, 5)
Ps = range(0, 2)
Qs = range(0, 2)

In [16]:
parameters = itertools.product(ps, qs, Ps, Qs)
parameters_list = list(parameters)
len(parameters_list)

In [17]:
residuals_results, models = find_best_arima(resids3.residuals.values[0:744*6], simple_dif=1, seson_diff=0, 
                                            parameters_list=parameters_list[80:])

In [18]:
print(sorted(residuals_results, key=lambda tup: tup[1], reverse=False)[:5])

best models

[[(3, 1, 1, 1), 46776.46106883851], [(2, 1, 1, 1), 46779.42063216587], [(1, 1, 1, 1), 46780.752699463446], [(1, 2, 1, 1), 46781.47510706796], [(1, 4, 1, 1), 46790.923233770605]]

##### best model (3, 1, 1, 1)

In [51]:
best_model = sm.tsa.statespace.SARIMAX(resids3.residuals.values, order=(3, 1, 1), 
                                        seasonal_order=(1, 0, 1, 24)).fit(disp=-1)

In [52]:
plt.figure(figsize=(15,8))
plt.subplot(211)
plt.plot(best_model.resid[1:])
plt.ylabel(u'Residuals')

ax = plt.subplot(212)
sm.graphics.tsa.plot_acf(best_model.resid[1:].squeeze(), lags=168, ax=ax)
plt.show();

##### plot predicted and real residuals for may 2014

In [53]:
prediction_real(best_model.fittedvalues, resids3.residuals, (0, 744), "May 2014", 'residuals', (15,5), 
                ('predict residuals', 'real residuals'))

In [54]:
plot_result(prediction(final_model_clust_3, components=87, feature_num=2, dataframe=three_clst), 
            best_model.fittedvalues, three_clst.taxi_call_num, 0, 745, "may_2014")
plot_result(prediction(final_model_clust_3, components=87, feature_num=2, dataframe=three_clst), 
            best_model.fittedvalues, three_clst.taxi_call_num, 13920, 14664, "december_2015")

##### For all regions from cluster 3 we've got linear regression model on 60 week (t=168), 15 annual (t=365*24), 12 hour (t=11) fourier componentsС‚ + ARIMA(3, 1, 1, 1) for regression residuals. 

##### 5) Cluster 4, region - column number 30 in taxi dataframe

In [55]:
four_clst = pd.DataFrame(taxi.iloc[:, 30].values, index = taxi.index, columns=["taxi_call_num"])
four_clst.head()

In [56]:
fig, axes = plt.subplots(figsize = (150,15))
axes.title.set_size(40)
four_clst.plot(color="green", title="timeserie", fontsize=25, ax = axes)
plt.show()
print(four_clst.shape)

#double click on image makes it bigger

In [57]:
month_len = np.array([0, 744, 720, 744])

for i, c in enumerate(["May", "June", "July"]):
    four_clst[int(sum(month_len[:i+1])) : int(sum(month_len[:i+2]))].plot(figsize=(20, 5), title=c, color="green", fontsize=16, 
                                                                          grid=True)
    plt.show()

In [58]:
plt.figure(figsize(15,10))
sm.tsa.seasonal_decompose(four_clst[:2208]).plot()
print("DickeyвЂ“Fuller test: p=%f" % sm.tsa.stattools.adfuller(four_clst.taxi_call_num[:2208].values)[1])

In [59]:
TT = 365 * 24
Tt = 7 * 24
tt = 1
time = np.arange(0, taxi.shape[0])
for w_week in range(1, 61):
    four_clst["sin_week_" + str(w_week)] = np.sin(2*np.pi*w_week*time/Tt) 
    four_clst["cos_week_" + str(w_week)] = np.cos(2*np.pi*w_week*time/Tt)
for w_year in range(1, 21):
    four_clst["sin_year_" + str(w_year)] = np.sin(2*np.pi*w_year*time/TT) 
    four_clst["cos_year_" + str(w_year)] = np.cos(2*np.pi*w_year*time/TT)
for w_day in range(1, 31):
    four_clst["sin_day_" + str(w_day)] = np.sin(2*np.pi*w_day*time/tt)
    four_clst["cos_day_" + str(w_day)] = np.cos(2*np.pi*w_day*time/tt) 

##### using find_best_fourier function we may consistently choose number of each fourier components - to get best score and minimum correlation with seasonal lags

In [60]:
find_best_fourier(30, ((60, 168), (20, 8760), (60, 1), ))

In [61]:
models = []
for W in range(1, 31):
    m5 = smf.ols('taxi_call_num ~ ' + ' + '.join(list(four_clst.columns)[1: 2*60+1]) + ' + ' + 
                 ' + '.join(list(four_clst.columns)[2*60 + 1: 2*60 + 1 + 2*20]) + 
                 ' + ' + ' + '.join(list(four_clst.columns)[2*60 + 1 + 2*20: 2*60 + 1 + 2*20 + 2*W]), data=four_clst)
    model = m5.fit(cov_type='HC1')
    models.append(model)

##### 60 components with week period (Tt=168), 20 year components (TT=36524) and 20 hour prediods (tt=1)

In [63]:
m5 = smf.ols('taxi_call_num ~ ' + ' + '.join(list(four_clst.columns)[1: 2*60+1]) + ' + ' + 
                                  ' + '.join(list(four_clst.columns)[2*60 + 1: 2*60 + 1 + 2*20]) 
                                + ' + ' +' + '.join(list(four_clst.columns)[2*60 + 1 + 2*20: 2*60 + 1 + 2*20 + 2*20])
                                , data=four_clst)
                                #+ ' + november_anom', data=four_clst)

final_model_clust_4 = m5.fit(cov_type='HC1')

##### 60 РЅРµРґРµР»СЊРЅС‹С…, 20 РіРѕРґРѕРІС‹С… Рё 20 С‡Р°СЃРѕРІС‹С… РєРѕРјРїРѕРЅРµРЅС‚

In [65]:
plt.figure(figsize=(20,5))
plt.plot(final_model_clust_4.resid)
plt.title("residual plot")
plt.xlabel('time (hour)')
plt.ylabel('residuals')
plt.grid()
plt.show()

In [66]:
resids4 = pd.DataFrame(final_model_clust_4.resid.values, index = [time], columns=["residuals"]).round(2)
resids4.head()

In [67]:
plot_autocorr(resids4.residuals, lag=168)

In [68]:
resids4["diff_1"] = resids4.residuals - resids4.residuals.shift(1)

In [69]:
plot_autocorr(resids4["diff_1"][1:], lag=168)

In [70]:
''' Autocorrelation - РїР°СЂР°РјРµС‚СЂС‹ q, Q '''
ps = range(0, 15)
qs = range(0, 8)
Ps = range(0, 2)
Qs = range(0, 2)

In [15]:
parameters = itertools.product(ps, qs, Ps, Qs)
parameters_list = list(parameters)
len(parameters_list)

In [18]:
residuals_results, models = find_best_arima(resids4.residuals.values[0:744*6], simple_dif=1, seson_diff=0, 
                                            parameters_list=parameters_list[400:])

In [19]:
print(sorted(residuals_results, key=lambda tup: tup[1], reverse=False)[:5])

best models

[[(9, 2, 1, 1), 47741.49595955429], [(8, 2, 1, 1), 47742.793918648335], [(14, 3, 1, 1), 47743.70836576415], [(6, 2, 1, 1), 47744.615492413956], [(4, 3, 1, 1), 47744.76752001122]]

##### best model: (9, 2, 1, 1)

In [71]:
best_model = sm.tsa.statespace.SARIMAX(resids4.residuals.values, order=(9, 1, 2), 
                                        seasonal_order=(1, 0, 1, 24)).fit(disp=-1)

In [25]:
taxi.columns[30]

In [72]:
plt.figure(figsize=(15,8))
plt.subplot(211)
plt.plot(best_model.resid[1:])#.plot()
plt.ylabel(u'Residuals')

ax = plt.subplot(212)
sm.graphics.tsa.plot_acf(best_model.resid[1:].squeeze(), lags=168, ax=ax)
plt.show();

##### plot predicted and real residuals for may 2014

In [73]:
prediction_real(best_model.fittedvalues, resids4.residuals, (0, 744), "May 2014", 'residuals', (15,5), 
                ('predict residuals', 'real residuals'))

In [74]:
plot_result(prediction(final_model_clust_4, components=100, feature_num=2, dataframe=four_clst), 
            best_model.fittedvalues, four_clst.taxi_call_num, 0, 745, "may_2014")
plot_result(prediction(final_model_clust_4, components=100, feature_num=2, dataframe=four_clst), 
            best_model.fittedvalues, four_clst.taxi_call_num, 13920, 14664, "december_2015")


##### For all regions from cluster 4 we've got linear regression model on 60 week (t=168), 20 annual (t=365*24), 20 hour (t=1) fourier componentsС‚ + ARIMA(9, 2, 1, 1) for regression residuals. Use this model to fit all cluster regions 

##### 6) Cluster 5, region - column number 1 in taxi dataframe

In [75]:
five_clst = pd.DataFrame(taxi.iloc[:, 1].values, index = taxi.index, columns=["taxi_call_num"])
five_clst.head()

In [76]:
fig, axes = plt.subplots(figsize = (150,15))
axes.title.set_size(40)
five_clst.plot(color="green", title="timeserie", fontsize=25, ax = axes)
plt.show()
print(five_clst.shape)

#double click on image makes it bigger

In [77]:
month_len = np.array([0, 744, 720, 744])

for i, c in enumerate(["May", "June", "July"]):
    five_clst[int(sum(month_len[:i+1])) : int(sum(month_len[:i+2]))].plot(figsize=(20, 5), title=c, color="green", fontsize=16, grid=True)
    plt.show()

In [78]:
plt.figure(figsize(15,10))
sm.tsa.seasonal_decompose(five_clst[:2208]).plot()
print("DickeyвЂ“Fuller test: p=%f" % sm.tsa.stattools.adfuller(five_clst.taxi_call_num[:2208].values)[1])

In [79]:
TT = 365 * 24
Tt = 7 * 24
tt = 1
time = np.arange(0, taxi.shape[0])
for w_week in range(1, 61):
    five_clst["sin_week_" + str(w_week)] = np.sin(2*np.pi*w_week*time/Tt) 
    five_clst["cos_week_" + str(w_week)] = np.cos(2*np.pi*w_week*time/Tt)
for w_day in range(1, 31):
    five_clst["sin_day_" + str(w_day)] = np.sin(2*np.pi*w_day*time/tt)
    five_clst["cos_day_" + str(w_day)] = np.cos(2*np.pi*w_day*time/tt) 

In [80]:
five_clst.shape

##### using find_best_fourier function we may consistently choose number of each fourier components - to get best score and minimum correlation with seasonal lags

In [81]:
find_best_fourier(1, ((60, 168), (60, 1), ))

##### 60 components with week period (Tt=168) and 12 hour prediods (tt=1)

In [83]:
m6 = smf.ols('taxi_call_num ~ ' + ' + '.join(list(five_clst.columns)[1: 2*60+1]) + ' + ' + 
                 ' + '.join(list(five_clst.columns)[2*60 + 1: 2*60 + 1 + 2*12]), data=five_clst)

final_model_clust_5 = m6.fit(cov_type='HC1')

In [84]:
plt.figure(figsize=(20,6))
plt.plot(final_model_clust_5.resid)
plt.title("May-June-July_2014")
plt.xlabel('time (hour)')
plt.ylabel('residuals')
plt.grid()
plt.show()

##### residuals

In [85]:
resids5 = pd.DataFrame(final_model_clust_5.resid.values, index = [time], columns=["residuals"]).round(2)
resids5.head()

In [86]:
plot_autocorr(resids5.residuals, lag=168)

In [87]:
resids5["diff_1"] = resids5.residuals - resids5.residuals.shift(1)

In [88]:
plot_autocorr(resids5["diff_1"][1:], lag=168)

In [89]:
''' Autocorrelation - РїР°СЂР°РјРµС‚СЂС‹ q, Q '''
ps = range(0, 14)
qs = range(0, 2)
Ps = range(0, 2)
Qs = range(0, 2)

In [16]:
parameters = itertools.product(ps, qs, Ps, Qs)
parameters_list = list(parameters)
len(parameters_list)

In [17]:
residuals_results, models = find_best_arima(resids5.residuals.values[0:744*6], simple_dif=1, seson_diff=0, 
                                            parameters_list=parameters_list[80:])

In [18]:
print(sorted(residuals_results, key=lambda tup: tup[1], reverse=False)[:5])

best models

[[(10, 1, 1, 1), 38552.39576565322], [(11, 1, 1, 1), 38552.702862086866], [(5, 1, 1, 1), 38554.870367152005], [(6, 1, 1, 1), 38555.352233191625], [(13, 1, 1, 1), 38555.85244020156]]

##### best model: (10, 1, 1, 1)

In [91]:
best_model = sm.tsa.statespace.SARIMAX(resids5.residuals.values, order=(11, 1, 1), 
                                        seasonal_order=(1, 0, 1, 24)).fit(disp=-1)

In [16]:
taxi.columns[1]

In [92]:
plt.figure(figsize=(15,8))
plt.subplot(211)
plt.plot(best_model.resid[1:])#.plot()
plt.ylabel(u'Residuals')

ax = plt.subplot(212)
sm.graphics.tsa.plot_acf(best_model.resid[1:].squeeze(), lags=168, ax=ax);

##### plot predicted and real residuals for may 2014

In [93]:
prediction_real(best_model.fittedvalues, resids5.residuals, (0, 744), "May 2014", 'residuals', (15,5), 
                ('predict residuals', 'real residuals'))

In [94]:
plot_result(prediction(final_model_clust_5, components=72, feature_num=2, dataframe=five_clst), 
            best_model.fittedvalues, five_clst.taxi_call_num, 0, 745, "may_2014")
plot_result(prediction(final_model_clust_5, components=72, feature_num=2, dataframe=five_clst), 
            best_model.fittedvalues, five_clst.taxi_call_num, 13920, 14664, "december_2015")

##### For all regions from cluster 5 we've got linear regression model on 60 week (t=168), 30 hour (t=1) and no annual fourier componentsС‚ + ARIMA(10, 1, 1, 1) for regression residuals. 

##### now we can use all selected parameters to fit training data and predict taxi calls in future

In [14]:
clusters = [[7, 11, 12, 13, 14, 15, 21, 27, 28, 31, 41, 61, 71, 72, 73, 84],
            [74, 76, 77, 79, 80, 81, 82, 83, 85, 87, 88, 91],
            [0, 3, 4, 5, 6, 17, 18, 19, 32, 33, 34, 35, 36, 44, 45, 95, 96, 97, 98, 99, 100, 101],
            [8, 16, 24, 25, 26, 29, 30, 38, 39, 40, 51, 52, 62, 63, 64, 65, 75, 89, 90, 92],
            [1, 2, 10, 20, 22, 23, 42, 43, 46, 47, 49, 50, 53, 54, 55, 56, 57, 58, 66, 67, 69, 78, 93, 94],
            [9, 37, 48, 59, 60, 68, 70, 86],
           ]

model_parameters = [(((168, 70), (8760, 15), (1, 12)), (5, 1, 1, 1, 1, 0)),
                    (((168, 70), (8760, 20), (1, 12)), (14, 1, 1, 1, 1, 0)),
                    (((168, 60), (8760, 15), (11, 12)), (3, 1, 1, 1, 1, 0)),
                    (((168, 60), (8760, 20), (1, 30)), (9, 2, 1, 1, 1, 0)),
                    (((168, 60), (1, 30)), (11, 1, 1, 1, 1, 0)),
                    (((168, 80), (8760, 15), (2, 23)), (13, 2, 1, 1, 1, 0)),
                   ]

##### get 2-year long fitted datafarme and frame with predictions: create empty frames - rows time, columns - regions

In [15]:
r_pred, c_red = verify_time.shape[0] + time.shape[0], data_2month.T[0, :].shape[0]

t_full = pd.date_range('2014 May 1 00:00:00', periods = data_2year.shape[1] +  data_2month.shape[1] - 2, freq = 'h')
f = lambda x: pd.DataFrame(np.zeros((r_pred, c_red)), index = [t_full], columns=[data_2month.T[0, :].astype(int)])

pred_taxi_regress, pred_taxi_arima1, pred_taxi_arima2 = f(1), f(1), f(1)

##### fill three dataframes with regression values and arima values "from past" and arima prediction

In [21]:
t_pred = pd.date_range('2014-05-01 00:00:00', '2016-06-07 23:00:00', freq = 'h')

for number, cluster in enumerate(clusters):
    for region in cluster:
        if region not in [7, 11, 12, 13, 14, 15, 21, 27, 28, 31, 41]:
            frame = make_prediction(regres=model_parameters[number][0], arima=model_parameters[number][1], region=region, 
                                    prediction_period=('2016-05-01 00:00:00', '2016-06-07 23:00:00'), draw=False)
            if isinstance(frame, pd.DataFrame):
                pred_taxi_regress.iloc[:t_pred.shape[0], region] = frame.regression_prediction.values
                pred_taxi_arima1.iloc[:t_pred.shape[0], region] = frame.average_resid_pred.values
                pred_taxi_arima2.iloc[:t_pred.shape[0], region] = frame.resid_prediction.values

In [53]:
pred_taxi_regress.round().to_csv("./data/week_4_regression", sep='\t', columns=list(pred_taxi_regress.columns))
pred_taxi_arima1.round().to_csv("./data/week_4_arima_average", sep='\t', columns=list(pred_taxi_arima1.columns))
pred_taxi_arima2.round().to_csv("./data/week_4_arima_predict", sep='\t', columns=list(pred_taxi_arima2.columns))

In [17]:
k = lambda name: pd.read_csv(name, sep='\t', index_col="Unnamed: 0")

reg_df, arima_av_df, arima_pred_df = k("./data/week_4_regression"), \
                                     k("./data/week_4_arima_average"), \
                                     k("./data/week_4_arima_predict")

In [56]:
reg_df.head()

##### now we may use r2_score from sklearn.metrics module to verify accuracy of our prediction

##### plot one arbitrary selected month - let it be april 2015 and calculated r2 value for each region

In [18]:
for reg in range(102):
    ''' select only train data '''
    fitted_value = reg_df.iloc[:17544, reg].values + arima_av_df.iloc[:17544, reg].values
    prediction_real(fitted_value, taxi.iloc[:, reg].values, (336*24, 365*24), 'april 2015 region: '+reg_df.columns[reg], 
                    "taxi calls", (15,5), ('prediction', 'real values')) 

In [78]:
r2_train = [r2_score(reg_df.iloc[:17544, reg].values + arima_av_df.iloc[:17544, reg].values, 
                     taxi.iloc[:17544, reg].values) 
            for reg in range(102)]

In [80]:
print("train r-2 score: ", r2_train)

##### plot predicted two weeks of may 2016 in two variants -  using arima prediction and "arima from past" - arima fitted values exact one year ago

In [91]:
for reg in range(102):
    ''' select only train data '''
    fitted_pred = reg_df.iloc[17544:17544+168*2, reg].values + arima_pred_df.iloc[17544:17544+168*2, reg].values
    fitted_av = reg_df.iloc[17544:17544+168*2, reg].values + arima_av_df.iloc[17544:17544+168*2, reg].values
    real = verify_taxi.iloc[0:168*2, reg].values
    plt.figure(figsize=(15,5))
    plt.plot(real, alpha = 0.7, label = 'real values')
    plt.plot(fitted_pred, alpha=0.5, label = 'prediction', color="red")
    plt.plot(fitted_av, alpha=0.5, label = 'prediction average arima', color="green")
    plt.xlabel('time (hours)')
    plt.ylabel("taxi calls")
    plt.title('first half may 2016 region: '+reg_df.columns[reg])
    plt.legend()
    plt.grid()  

# Part 2 supplementary materials

##### try to choose correct number of clusters using special clustering method  

##### http://nbviewer.jupyter.org/github/alexminnaar/time-series-classification-and-clustering/blob/master/Time%20Series%20Classification%20and%20Clustering.ipynb      <br> <br> https://github.com/alexminnaar/time-series-classification-and-clustering

In [9]:
class ts_cluster(object):
    def __init__(self,num_clust=100):
        '''
        num_clust is the number of clusters for the k-means algorithm
        assignments holds the assignments of data points (indices) to clusters
        centroids holds the centroids of the clusters
        '''
        self.num_clust=num_clust
        self.assignments={}
        self.centroids=[]
    
    def compa_clust(self,s1,centroid,w):        
        centroid_part = centroid[:,:5]
        self.assign = pd.Series([[] for i in range(len(centroid))],index=np.arange(len(centroid)))

        for ind,i in enumerate(s1):
            min_dist=float('inf')
            #closest_clust=None
            for c_ind,j in enumerate(centroid_part):
                    
                if self.LB_Keogh(i,j,5)<min_dist:
                    cur_dist=self.SpatioTemporalDis(i, j, w)
                        
                    if cur_dist<min_dist:
                        min_dist=cur_dist
                        closest_clust=c_ind
            
            
            self.assign[closest_clust].append(ind)

        print(self.assign)
        #print s1[1]
        self.s2 = pd.Series([[] for i in range(len(s1))],index=np.arange(len(s1)))
        #print self.s2
        for key in self.assign.index:
            for k in self.assign[key]:
                self.s2[k] = centroid[key,6:].tolist()
                #print self.s2[k]
        self.s2 = np.array(self.s2)
        return self.s2
        
    
    
    def k_means_clust(self,data,num_iter,w,progress=True):
        
        '''
        k-means clustering algorithm for time series data.  dynamic time warping Euclidean distance
         used as default similarity measure. 
        '''
        self.centroids=random.sample(list(data),self.num_clust)
        #print(len(self.centroids))
        for n in range(num_iter):
            if progress:
                print('iteration '+str(n+1), end=' ')
            
            self.assignments={}
            for ind,i in enumerate(data):
                min_dist=float('inf')
                #closest_clust=None
                for c_ind,j in enumerate(self.centroids):
                    if self.LB_Keogh(i,j,5)<min_dist:
                        cur_dist=self.SpatioTemporalDis(i, j, w)
                        
                        if cur_dist<min_dist:
                            min_dist=cur_dist
                            closest_clust=c_ind
                
                if closest_clust in self.assignments:
                    self.assignments[closest_clust].append(ind)
                else:
                    self.assignments[closest_clust]=[]
                    
            #print(len(self.assignments))
            #recalculate centroids of clusters
            for key in self.assignments:
                clust_sum=0
                for k in self.assignments[key]:
                    clust_sum=clust_sum+data[k]
                self.centroids[key]=[m/len(self.assignments[key]) for m in clust_sum]
            
    def get_centroids(self):
        return self.centroids
        
    def get_assignments(self):
        return self.assignments
        
    def plot_centroids(self):
        for i in self.centroids:
            plt.plot(i)
        plt.show()
        
    def SpatioTemporalDis(self,s1,s2,w):
        
        f1,f2 = s1[0:2],s2[0:2]
        v1,v2 = s1[2:],s2[2:]
        disInvari = self.InvarDistance(f1, f2)
        disVari   = self.DTWDistance(v1, v2, w)
        
        return np.sqrt(disInvari+disVari)
    
    
    def InvarDistance(self,f1,f2):
        '''calculates the invariant features distance using Euclidean distance'''
         
        dis = spatial.distance.euclidean(f1, f2)
        
        return dis
        
    def DTWDistance(self,s1,s2,w=None):
        '''
        Calculates dynamic time warping Euclidean distance between two
        sequences. Option to enforce locality constraint for window w.
        '''
        DTW={}
        
        if w:
            w = max(w, abs(len(s1)-len(s2)))
    
            for i in range(-1,len(s1)):
                for j in range(-1,len(s2)):
                    DTW[(i, j)] = float('inf')
            
        else:
            for i in range(len(s1)):
                DTW[(i, -1)] = float('inf')
            for i in range(len(s2)):
                DTW[(-1, i)] = float('inf')
        
        DTW[(-1, -1)] = 0
        
        for i in range(len(s1)):
            if w:
                for j in range(max(0, i-w), min(len(s2), i+w)):
                    dist= (s1[i]-s2[j])**2
                    DTW[(i, j)] = dist + min(DTW[(i-1, j)],DTW[(i, j-1)], DTW[(i-1, j-1)])
            else:
                for j in range(len(s2)):
                    dist= (s1[i]-s2[j])**2
                    DTW[(i, j)] = dist + min(DTW[(i-1, j)],DTW[(i, j-1)], DTW[(i-1, j-1)])
            
        return (DTW[len(s1)-1, len(s2)-1])      
        
    def LB_Keogh(self,s1,s2,r):
        '''
        Calculates LB_Keough lower bound to dynamic time warping. Linear
        complexity compared to quadratic complexity of dtw.
        '''
        LB_sum=0
        for ind,i in enumerate(s1):
            
            lower_bound=min(s2[(ind-r if ind-r>=0 else 0):(ind+r)])
            upper_bound=max(s2[(ind-r if ind-r>=0 else 0):(ind+r)])
            
            if i>upper_bound:
                LB_sum=LB_sum+(i-upper_bound)**2
            elif i<lower_bound:
                LB_sum=LB_sum+(i-lower_bound)**2
            
        return np.sqrt(LB_sum)

##### may choose number of clusters, number of iterations and size of window w: trying different values - lets take number of iterations = 20, window size = 5 (default value)

In [26]:
clust_obj = ts_cluster(num_clust=4)
clust_obj.k_means_clust(taxi_data.T.values[:, :744],num_iter=20,w=5,progress=True)

In [11]:
#clust_obj.get_centroids()
#clust_obj.get_assignments()
#list(clust_obj.get_assignments().values())

In [33]:
def get_centr_label(clust_obj, num_clust):
    cluster_centroids = clust_obj.get_centroids()
    cluster_labels = np.array([num_clust]*102)
    for i in range(cluster_labels.shape[0]):
        label = [key for key, values in clust_obj.get_assignments().items() if i in values]
        if len(label) > 0:
            cluster_labels[i] = label[0]
    return cluster_centroids, cluster_labels       

In [36]:
four_clust_centroids, four_clust_labels = get_centr_label(clust_obj_more, num_clust=4)

In [39]:
four_clust_labels

##### plot cluster centroids

In [42]:
clust_obj.plot_centroids()

In [90]:
def plot_centroid(clust_obj, plt_num=4):
    f, axarr = plt.subplots(int(np.ceil(plt_num / 3.0)), 3)
    f.set_size_inches(15, 10)
    for i in range(plt_num):
        axarr[i//3, i%3].plot(clust_obj.get_centroids()[i])
        axarr[i//3, i%3].set_title('cluster: ' + str(i))
    f.subplots_adjust(hspace=0.3)
    plt.show()

In [47]:
plot_centroid(clust_obj)

In [109]:
plot_silhouette_coeff(num_clusters=4, cluster_labels=four_clust_labels)

##### try to choose different cluster numbers from 5 to 10 - do it for first month of time serie

In [105]:
taxi_data.T.values[:, :744].shape

In [80]:
clusters = []
for clst_num in range(5, 11):
    clusters.append(ts_cluster(num_clust=clst_num))
    clusters[-1].k_means_clust(taxi_data.T.values[:, :744],num_iter=20,w=5,progress=True)

In [62]:
for clst_num, clst_obj in enumerate(clusters):
    _, clust_labels = get_centr_label(clst_obj, num_clust=5+clst_num)
    plot_silhouette_coeff(num_clusters=5+clst_num, cluster_labels=clust_labels)

##### On my opinion n_clusters = 6 looks the best. Clusterization method use not euclidean metrics but silouette coeff calculation utilize it. 

## This way number of clusters equal to 6 was choosen
##### for each cluster print "bad" series with negative silouette

In [150]:
clust_centroids, clust_labels = get_centr_label(clusters[1], num_clust=5+clst_num)
bad_elements = plot_silhouette_coeff(6, clust_labels)
print(bad_elements)

##### look at each cluster centroids

In [92]:
plot_centroid(clusters[1], 6)

In [94]:
clusters[1].get_assignments().values()

##### plot all time series. "good"regions are plotted in blue color ("+" silouette coeffisient), "bad" region with negative coefficient are plotted in green color

In [157]:
for clst_num, cluster in enumerate(list(clusters[1].get_assignments().values())):
    f, axarr = plt.subplots(int(np.ceil(len(cluster) / 3.0)), 3)
    f.set_size_inches(15, 5*int(np.ceil(len(cluster) / 3.0)))
    for i in range(len(cluster)):
        if cluster[i] in bad_elements[clst_num]:
            axarr[i//3, i%3].plot(taxi_data.T.values[cluster[i], :744], color="green")
        else:
            axarr[i//3, i%3].plot(taxi_data.T.values[cluster[i], :744])
        axarr[i//3, i%3].set_title('cluster: ' + str(clst_num) + " region index: " + str(cluster[i]))
    f.subplots_adjust(hspace=0.3)
    plt.show()

In [161]:
print(sorted(list(itertools.chain.from_iterable(clusters[1].get_assignments().values()))))